In [1]:
from src.trainers import compile_and_fit_with_pinball_loss
from src.loaders.window_generator import WindowGenerator
from src.loaders.data_loader import load_basic_preprocessed_train, load_submission_data
from src.make_submissions.load_test_features import make_submission_df, to_submission_csv

import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [2]:
from src.loaders.data_loader import load_train_data
from src.preprocessors.preprocessors import split_train_valid_test

df = load_train_data()
df


,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [3]:
train_df, valid_df, test_df = split_train_valid_test(df, [0.7, 0.2, 0.1])

train_df_target = train_df["TARGET"]
valid_df_target = valid_df["TARGET"]
test_df_target = test_df["TARGET"]

train_df, valid_df, test_df = load_basic_preprocessed_train("minmax")

train_df["scaled_TARGET"] = train_df["TARGET"]
valid_df["scaled_TARGET"] = valid_df["TARGET"]
test_df["scaled_TARGET"] = test_df["TARGET"]

train_df["TARGET"] = train_df_target
valid_df["TARGET"] = valid_df_target
test_df["TARGET"] = test_df_target

train_df.describe()

ratio should be 3 length list, example: [0.6, 0.3, 0.1]
sum of ration should be 1, example: [0.7, 0.2, 0.1]
shape of train, valid, test: (36792, 9), (10511, 9), (5257, 9)
ratio should be 3 length list, example: [0.6, 0.3, 0.1]
sum of ration should be 1, example: [0.7, 0.2, 0.1]
shape of train, valid, test: (36792, 18), (10511, 18), (5257, 18)


,DHI,DNI,WS,RH,T,TARGET,Day_sin,Day_cos,Hour_sin,Hour_cos,GHI,TARGET_ROLLING_MEAN_3_shift_1,TARGET_ROLLING_MEAN_5_shift_2,TARGET_ROLLING_MEAN_11_shift_5,TARGET_ROLLING_MEAN_23_shift_11,TARGET_ROLLING_MEAN_47_shift_23,GHI_ANGLE_COS,GHI_ANGLE_SIN,scaled_TARGET
count,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000,3.679200e+04,36792.000000,3.679200e+04,36792.000000,36792.000000,36792.000000,36792.000000,36792.000000
mean,0.119412,0.219731,0.201327,0.542351,0.512100,17.439978,0.507046,0.522326,0.500206,0.500027,0.256946,1.754282e-01,0.177078,1.873781e-01,0.239824,0.469365,0.717934,0.497078,0.174550
std,0.194232,0.330292,0.117052,0.234989,0.192822,25.449577,0.346981,0.359263,0.353558,0.353558,0.340088,2.493108e-01,0.245816,2.357651e-01,0.221283,0.211497,0.250734,0.163514,0.254715
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.116667,0.364896,0.351852,0.000000,0.159610,0.158025,0.146447,0.146447,0.000000,3.356260e-15,0.000000,1.852145e-16,0.044158,0.297058,0.595459,0.480000,0.000000
50%,0.000000,0.000000,0.183333,0.553620,0.500000,0.000000,0.530107,0.536539,0.500000,0.500000,0.000000,1.006823e-02,0.023250,7.214131e-02,0.186432,0.447393,0.879112,0.480000,0.000000
75%,0.160985,0.430831,0.258333,0.711963,0.648148,31.085053,0.840390,0.891488,0.853553,0.853553,0.557776,3.152806e-01,0.319378,3.256305e-01,0.378664,0.634329,0.879112,0.480000,0.311118
max,1.000000,1.000000,1.000000,1.000000,1.000000,99.913939,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
cutter = [
    'DHI', 
#     'DNI', 
#     'WS', 
#     'RH', 
#     'T', 
#     'Day_sin', 
#     'Day_cos',
#       Hour_sin', 
#     'Hour_cos', 
    'GHI', 
    'TARGET_ROLLING_MEAN_3_shift_1',
    'TARGET_ROLLING_MEAN_5_shift_2', 
    'TARGET_ROLLING_MEAN_11_shift_5',
    'TARGET_ROLLING_MEAN_23_shift_11', 
    'TARGET_ROLLING_MEAN_47_shift_23',
    'GHI_ANGLE_COS', 
    'GHI_ANGLE_SIN', 
    'scaled_TARGET',
    "TARGET",
]
train_df = train_df[cutter]
valid_df = valid_df[cutter]
test_df = test_df[cutter]

In [5]:
TAU = 0.5
ONE_DAY_STEPS = 48
OUTPUT_STEPS = ONE_DAY_STEPS * 2

In [10]:
import src.model.multiple_output.convolution
import importlib
importlib.reload(src.model.multiple_output.convolution)

from src.model.multiple_output.convolution import ConvolutionVarious
from src.loaders.data_loader import load_test_features, load_basic_preprocessed_predict

one_days_window_label_columns = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=ONE_DAY_STEPS * 1,
    label_width=OUTPUT_STEPS,
    shift=OUTPUT_STEPS,
    sequence_stride=1,
    label_columns=["TARGET"]
    
)
submission_df = load_submission_data()
_, predict_df = load_basic_preprocessed_predict()
predict_df["scaled_TARGET"] = predict_df["TARGET"]
predict_df = predict_df[cutter]
predict_df.drop("TARGET", axis=1, inplace=True)
predict_df = load_test_features(predict_df, 48 * 1)

evaluate_dict = {}

for i in range(1, 10):
    q = i/10
    print(q)
    
    conv_various = ConvolutionVarious(48)

    compile_and_fit_with_pinball_loss(conv_various, one_days_window_label_columns, q)
    evaluate_dict[q] = conv_various.evaluate(one_days_window_label_columns.test, verbose=0)
    predict_np = predict_df.reshape(-1, 48, train_df.columns.shape[0] - 1)
    pred_y = conv_various.predict(predict_np)[:, :, -1]
    submission_df[f"q_{q}"] = pred_y.reshape(-1)

to_submission_csv(submission_df, "conv_various_dhi_dni_ghi_standard_scaled_n_minmaxed_zero_1.csv")

ratio should be 3 length list, example: [0.6, 0.3, 0.1]
sum of ration should be 1, example: [0.7, 0.2, 0.1]
shape of train, valid, test: (36792, 18), (10511, 18), (5257, 18)
0.1
Epoch 1/1000
1146/1146 [==============================] - 12s 10ms/step - loss: 1.7470 - mean_absolute_error: 17.4690 - mean_squared_error: 953.9720 - _pinball_loss: 1.7470 - val_loss: 2.1811 - val_mean_absolute_error: 21.8107 - val_mean_squared_error: 1310.7937 - val__pinball_loss: 2.1811
Epoch 2/1000
1146/1146 [==============================] - 12s 11ms/step - loss: 1.7469 - mean_absolute_error: 17.4691 - mean_squared_error: 954.1145 - _pinball_loss: 1.7469 - val_loss: 2.1811 - val_mean_absolute_error: 21.8107 - val_mean_squared_error: 1310.7938 - val__pinball_loss: 2.1811
Epoch 3/1000
1146/1146 [==============================] - 12s 11ms/step - loss: 1.7464 - mean_absolute_error: 17.4636 - mean_squared_error: 953.6229 - _pinball_loss: 1.7464 - val_loss: 2.1811 - val_mean_absolute_error: 21.8107 - val_mean_sq

Epoch 21/1000
1146/1146 [==============================] - 11s 10ms/step - loss: 2.4455 - mean_absolute_error: 4.9610 - mean_squared_error: 107.6167 - _pinball_loss: 2.4455 - val_loss: 3.1431 - val_mean_absolute_error: 6.3615 - val_mean_squared_error: 158.8558 - val__pinball_loss: 3.1431
Epoch 22/1000
1146/1146 [==============================] - 11s 10ms/step - loss: 2.4396 - mean_absolute_error: 4.9472 - mean_squared_error: 107.3369 - _pinball_loss: 2.4396 - val_loss: 3.1401 - val_mean_absolute_error: 6.3734 - val_mean_squared_error: 158.4314 - val__pinball_loss: 3.1401
Epoch 23/1000
1146/1146 [==============================] - 11s 9ms/step - loss: 2.4333 - mean_absolute_error: 4.9313 - mean_squared_error: 106.9356 - _pinball_loss: 2.4333 - val_loss: 3.1284 - val_mean_absolute_error: 6.3834 - val_mean_squared_error: 158.4094 - val__pinball_loss: 3.1284
Epoch 24/1000
1146/1146 [==============================] - 11s 9ms/step - loss: 2.4276 - mean_absolute_error: 4.9196 - mean_squared_er

1146/1146 [==============================] - 22s 19ms/step - loss: 2.3422 - mean_absolute_error: 4.6844 - mean_squared_error: 109.8982 - _pinball_loss: 2.3422 - val_loss: 3.0365 - val_mean_absolute_error: 6.0731 - val_mean_squared_error: 162.4495 - val__pinball_loss: 3.0365
Epoch 25/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.3347 - mean_absolute_error: 4.6693 - mean_squared_error: 109.7433 - _pinball_loss: 2.3347 - val_loss: 3.0535 - val_mean_absolute_error: 6.1070 - val_mean_squared_error: 161.4160 - val__pinball_loss: 3.0535
Epoch 26/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.3315 - mean_absolute_error: 4.6630 - mean_squared_error: 109.4599 - _pinball_loss: 2.3315 - val_loss: 3.0267 - val_mean_absolute_error: 6.0534 - val_mean_squared_error: 161.3115 - val__pinball_loss: 3.0267
Epoch 27/1000
1146/1146 [==============================] - 23s 20ms/step - loss: 2.3286 - mean_absolute_error: 4.6572 - mean_squared_error: 109.384

Epoch 6/1000
1146/1146 [==============================] - 27s 23ms/step - loss: 2.3342 - mean_absolute_error: 5.1961 - mean_squared_error: 131.8151 - _pinball_loss: 2.3342 - val_loss: 2.9577 - val_mean_absolute_error: 6.6156 - val_mean_squared_error: 197.0029 - val__pinball_loss: 2.9577
Epoch 7/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.2982 - mean_absolute_error: 5.1266 - mean_squared_error: 130.7213 - _pinball_loss: 2.2982 - val_loss: 2.9249 - val_mean_absolute_error: 6.5163 - val_mean_squared_error: 192.5920 - val__pinball_loss: 2.9249
Epoch 8/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.2738 - mean_absolute_error: 5.0808 - mean_squared_error: 130.0929 - _pinball_loss: 2.2738 - val_loss: 2.9032 - val_mean_absolute_error: 6.4684 - val_mean_squared_error: 190.6518 - val__pinball_loss: 2.9032
Epoch 9/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.2514 - mean_absolute_error: 5.0356 - mean_squared_erro

Epoch 4/1000
1146/1146 [==============================] - 21s 19ms/step - loss: 2.2395 - mean_absolute_error: 6.0912 - mean_squared_error: 157.9425 - _pinball_loss: 2.2395 - val_loss: 2.7343 - val_mean_absolute_error: 7.4826 - val_mean_squared_error: 228.8010 - val__pinball_loss: 2.7343
Epoch 5/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.1697 - mean_absolute_error: 5.9420 - mean_squared_error: 155.6721 - _pinball_loss: 2.1697 - val_loss: 2.6728 - val_mean_absolute_error: 7.1933 - val_mean_squared_error: 218.9552 - val__pinball_loss: 2.6728
Epoch 6/1000
1146/1146 [==============================] - 21s 19ms/step - loss: 2.1202 - mean_absolute_error: 5.8356 - mean_squared_error: 154.1689 - _pinball_loss: 2.1202 - val_loss: 2.6182 - val_mean_absolute_error: 7.1871 - val_mean_squared_error: 219.1118 - val__pinball_loss: 2.6182
Epoch 7/1000
1146/1146 [==============================] - 22s 19ms/step - loss: 2.0770 - mean_absolute_error: 5.7495 - mean_squared_erro

Epoch 18/1000
1146/1146 [==============================] - 25s 22ms/step - loss: 1.3790 - mean_absolute_error: 5.8205 - mean_squared_error: 174.7696 - _pinball_loss: 1.3790 - val_loss: 1.7425 - val_mean_absolute_error: 7.3557 - val_mean_squared_error: 256.2310 - val__pinball_loss: 1.7425
Epoch 19/1000
1146/1146 [==============================] - 30s 26ms/step - loss: 1.3718 - mean_absolute_error: 5.7975 - mean_squared_error: 174.4981 - _pinball_loss: 1.3718 - val_loss: 1.7348 - val_mean_absolute_error: 7.2691 - val_mean_squared_error: 252.4974 - val__pinball_loss: 1.7348
Epoch 20/1000
1146/1146 [==============================] - 25s 22ms/step - loss: 1.3612 - mean_absolute_error: 5.7673 - mean_squared_error: 174.1539 - _pinball_loss: 1.3612 - val_loss: 1.7494 - val_mean_absolute_error: 7.2389 - val_mean_squared_error: 253.0397 - val__pinball_loss: 1.7494
Epoch 21/1000
1146/1146 [==============================] - 38s 34ms/step - loss: 1.3550 - mean_absolute_error: 5.7451 - mean_squared_

Epoch 10/1000
1146/1146 [==============================] - 11s 9ms/step - loss: 0.8952 - mean_absolute_error: 7.4990 - mean_squared_error: 223.4914 - _pinball_loss: 0.8952 - val_loss: 1.0888 - val_mean_absolute_error: 8.9872 - val_mean_squared_error: 314.2141 - val__pinball_loss: 1.0888
Epoch 11/1000
1146/1146 [==============================] - 11s 9ms/step - loss: 0.8798 - mean_absolute_error: 7.3826 - mean_squared_error: 220.9973 - _pinball_loss: 0.8798 - val_loss: 1.0821 - val_mean_absolute_error: 8.5782 - val_mean_squared_error: 299.2605 - val__pinball_loss: 1.0821
Epoch 12/1000
1146/1146 [==============================] - 11s 9ms/step - loss: 0.8690 - mean_absolute_error: 7.3113 - mean_squared_error: 219.6607 - _pinball_loss: 0.8690 - val_loss: 1.0697 - val_mean_absolute_error: 8.7821 - val_mean_squared_error: 309.5882 - val__pinball_loss: 1.0697
Epoch 13/1000
1146/1146 [==============================] - 12s 10ms/step - loss: 0.8593 - mean_absolute_error: 7.2557 - mean_squared_err

1146/1146 [==============================] - 10s 9ms/step - loss: 0.7283 - mean_absolute_error: 6.3806 - mean_squared_error: 201.7129 - _pinball_loss: 0.7283 - val_loss: 0.9635 - val_mean_absolute_error: 8.1751 - val_mean_squared_error: 297.2507 - val__pinball_loss: 0.9635
Epoch 67/1000
1146/1146 [==============================] - 10s 9ms/step - loss: 0.7254 - mean_absolute_error: 6.3606 - mean_squared_error: 201.2149 - _pinball_loss: 0.7254 - val_loss: 0.9595 - val_mean_absolute_error: 8.1381 - val_mean_squared_error: 295.7571 - val__pinball_loss: 0.9595
Epoch 68/1000
1146/1146 [==============================] - 11s 10ms/step - loss: 0.7266 - mean_absolute_error: 6.3680 - mean_squared_error: 201.4987 - _pinball_loss: 0.7266 - val_loss: 0.9619 - val_mean_absolute_error: 7.8339 - val_mean_squared_error: 280.7706 - val__pinball_loss: 0.9619
Epoch 69/1000
1146/1146 [==============================] - 11s 10ms/step - loss: 0.7246 - mean_absolute_error: 6.3509 - mean_squared_error: 200.8684 

In [12]:
s = 0
for i, v in evaluate_dict.items():
    s += v[0]
    
s / 9

1.6027346120940313

In [16]:
submission_df[0:10]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,-0.002159,-0.220615,-1.261153,9.486985,113.883850,34.812424,-13.457970,-28.959330,-53.813503
1,0.csv_Day7_0h30m,-0.373913,-1.051055,0.297321,-17.221001,23.366230,58.310581,94.569389,-21.290644,9.520831
2,0.csv_Day7_1h00m,-0.461657,-0.853200,-0.102798,-24.393742,-3.662500,-22.757494,146.346939,-26.589853,67.072571
3,0.csv_Day7_1h30m,0.108762,-0.634351,-0.526419,-23.442234,7.710019,-21.596439,48.220638,-54.732620,90.911415
4,0.csv_Day7_2h00m,0.340255,-0.692587,0.335839,-9.392648,-4.039066,-52.336159,-45.584362,-99.603485,84.718452
5,0.csv_Day7_2h30m,-0.748636,0.336466,-0.631239,5.342808,-54.227745,10.964619,-263.081207,-127.030037,15.754553
6,0.csv_Day7_3h00m,-0.103655,0.270761,-0.447056,1.126039,-59.988155,21.925920,-317.657104,-77.831413,11.285848
7,0.csv_Day7_3h30m,-0.667602,0.052774,0.504167,-5.856567,-20.215286,-58.455414,-287.889038,5.089137,62.255196
8,0.csv_Day7_4h00m,0.057454,-0.283889,-0.974525,-7.913906,7.009651,-177.984634,-171.105545,120.941635,185.078003
9,0.csv_Day7_4h30m,0.362557,-0.052823,-0.125718,-13.690651,23.123959,-264.648163,-64.478371,194.155914,285.917023
